In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import ConcatDataset, DataLoader, Subset
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from Helpers.data import SDDataset, PointCloudDataset
# Hyperparameters
input_dim = 3
latent_dim = 512
hidden_dim = 256
learning_rate = 0.001
num_epochs = 100

if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Using: cuda


In [2]:
d = SDDataset('../Data/ModelNet40', '../Data/sampled_points', device, 1024, 'train')

In [3]:
torch.save(d, '../Data/sd_dataset_1024_train.pt')

In [4]:
train = torch.load('../Data/sd_dataset_1024_train.pt', weights_only= False)

In [5]:
loader = DataLoader(train, batch_size= 32, shuffle= True)

for data in loader:
    print(data)
    latent_rep = data['latent_rep']
    xyz = data['xyz']
    sd = data['sd']

    print(latent_rep.shape)
    print(xyz.shape)
    print(sd.shape)

    break


{'latent_rep': tensor([[ 0.2781,  1.3495, -2.4532,  ..., -1.6509,  1.0050,  0.9629],
        [-1.4604, -1.6223, -0.8180,  ..., -1.1405,  1.4749,  0.9913],
        [ 0.1813, -0.9574,  3.0082,  ...,  0.9539, -0.4228, -0.7308],
        ...,
        [ 1.0613,  0.4580,  2.8835,  ..., -0.0247, -1.4942, -0.0771],
        [-3.1888,  1.4808,  1.0790,  ...,  0.9437,  0.5125, -0.3939],
        [ 0.4147,  1.0751, -1.5592,  ..., -1.1616, -0.1255, -0.3073]]), 'xyz': tensor([[ 0.7061,  0.7534,  0.1379],
        [ 0.5532,  0.5239,  0.3750],
        [ 0.9893,  0.1847,  0.4485],
        [ 0.2668,  0.3896,  0.8276],
        [ 0.7598,  0.3015,  0.3044],
        [-0.0119,  0.0950,  0.2527],
        [ 0.3906,  0.4128,  0.5396],
        [ 0.6807,  0.0923,  0.6934],
        [ 0.6528,  0.9932,  0.4612],
        [ 0.5547,  0.8198,  0.5054],
        [ 0.0927,  0.4495,  0.1548],
        [ 0.4529,  0.0330,  0.3574],
        [ 0.3962,  0.2510,  0.1333],
        [ 0.2583,  0.1052,  0.2367],
        [ 0.9038,  0.3157